In [1]:
import matplotlib.pyplot as plt
import cv2
from glob import glob 
import os
from ultralytics import YOLO
import ultralytics
import random

### 데이터 수집

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("detection-6zqm5").project("-ix50o")
version = project.version(8)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.27, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to 분리수거-프로젝트-8 in yolov8:: 100%|██████████| 13436/13436 [00:05<00:00, 2534.28it/s]


로보플로우에서 작업을 진행하여 약 2300장의 사진에 대해서 라벨링 작업을 수행 하였습니다.

- 로보플로우에서 사진 데이터의 픽셀 크기와 사이즈를 맞춰서 진행
- 90도 180도 360도로 회전을 가하여 추가적인 데이터 수집
- 채도 +-30%, 노이즈 발생의 조건을 추가적으로 가하여

총 5300장의 데이터를 학습 시키는데 사용 하였습니다.

In [3]:
# 커널 충돌 방지 코드
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [9]:

# # yolo 테스트 파일 다운로드 및 모델 다운로드 코드
# from ultralytics import YOLO

# ultralytics.checks()
# model = YOLO('yolov8n.pt')
# # model.predict(
# #    source='https://media.roboflow.com/notebooks/examples/dog.jpeg',
# #    conf=0.25
# # )

#### 모델 학습

In [5]:
model = YOLO('yolov8n.pt')
print(type(model.names), len(model.names))

<class 'dict'> 80


In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="TP8PbdzFturXX4hloYwF")
project = rf.workspace("detection-6zqm5").project("-ix50o")
version = project.version(4)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.27, to fix: `pip install ultralytics==8.0.196`


### 하이퍼 파라메터 찾기

In [3]:
from ultralytics import YOLO

# Initialize the YOLO model
model = YOLO('yolov8n.pt')

model.tune(data='data.yaml', epochs=10, iterations=100, optimizer='AdamW', plots=True, save=False, val=True)

Tuner: Initialized Tuner instance with 'tune_dir=runs\detect\tune'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/100 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}
Saved runs\detect\tune\tune_scatter_plots.png
Saved runs\detect\tune\tune_fitness.png

Tuner: 1/100 iterations complete ✅ (553.63s)
Tuner: Results saved to runs\detect\tune
Tuner: Best fitness=0.31936 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.30909, 'metrics/recall(B)': 0.52553, 'metrics/mAP50(B)': 0.38295, 'metrics/mAP50-95(B)': 0.31229, 'val/box_loss': 0.7958, 'val/cls_loss': 1.59948, 'va

In [6]:
model.train(data='data.yaml', epochs=100, patience=24, batch=24, imgsz=640, optimizer = 'AdamW',
            lr0=0.00145, # 초기 학습률
            lrf=0.00444,  # 최종 학습률
            momentum = 0.76895,
            weight_decay= 0.0006,
            warmup_epochs = 2.39428,
            warmup_momentum = 0.77794,
            box = 4.845,
            hsv_h= 0.01191,
            hsv_s= 0.45532,
            hsv_v= 0.28084,
            translate = 0.06495,
            mosaic = 0.56935,
            dropout=0.10, # 과적합 방지 10프로의 뉴런을 끄고 학습
            close_mosaic=0, # 오류 방지 해당 기능을 키면 학습중에 멈춤
            seed=13)

New https://pypi.org/project/ultralytics/8.1.40 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.27 🚀 Python-3.10.13 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=100, time=None, patience=24, batch=24, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=13, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False

train: Scanning C:\Users\HoJin\Desktop\python\ML_project\zreo_py\DL_Project\Trash-classification\ecosort\train\labels... 5868 images, 568 backgrounds, 0 corrupt: 100%|██████████| 5868/5868 [00:02<00:00, 2620.26it/s]


train: New cache created: C:\Users\HoJin\Desktop\python\ML_project\zreo_py\DL_Project\Trash-classification\ecosort\train\labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 8270, len(boxes) = 8576. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


val: Scanning C:\Users\HoJin\Desktop\python\ML_project\zreo_py\DL_Project\Trash-classification\ecosort\valid\labels... 565 images, 51 backgrounds, 0 corrupt: 100%|██████████| 565/565 [00:00<00:00, 1362.74it/s]


val: New cache created: C:\Users\HoJin\Desktop\python\ML_project\zreo_py\DL_Project\Trash-classification\ecosort\valid\labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 779, len(boxes) = 810. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
Plotting labels to runs\detect\train\labels.jpg... 
optimizer: AdamW(lr=0.00145, momentum=0.76895) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.000675), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        565        810      0.902      0.818      0.881      0.766
Stopping training early as no improvement observed in last 24 epochs. Best results observed at epoch 65, best model saved as best.pt.
To update EarlyStopping(patience=24) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



89 epochs completed in 1.066 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.3MB

Validating runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.1.27 🚀 Python-3.10.13 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 168 layers, 3007013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:02<00:00,  4.01it/s]


                   all        565        810      0.863      0.835      0.891      0.771
               General        565        165       0.73      0.656      0.727      0.541
                 Glass        565         97      0.943      0.948      0.968      0.873
                 Metal        565        116      0.945      0.885      0.946      0.847
                 Paper        565        184      0.913        0.8      0.912      0.857
               Plastic        565         95      0.939      0.863      0.951      0.845
             Styrofoam        565         42      0.653      0.857       0.82      0.675
                 Vinyl        565        111      0.917      0.838       0.91       0.76
Speed: 0.3ms preprocess, 1.7ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002993C7EDC00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047

In [5]:
print(type(model.names), len(model.names))

<class 'dict'> 7


In [6]:
results = model.predict(source='test/images/', save=True)


image 1/225 c:\Users\HoJin\Desktop\python\ML_project\zreo_py\DL_Project\Trash-classification\ecosort\test\images\-10_png.rf.ca08aef0e8f9d37d91f824809a55dea5.jpg: 640x640 1 Vinyl, 24.2ms
image 2/225 c:\Users\HoJin\Desktop\python\ML_project\zreo_py\DL_Project\Trash-classification\ecosort\test\images\-2_jpg.rf.38f87f357926e771563db44f60d7ad1a.jpg: 640x640 1 Metal, 16.4ms
image 3/225 c:\Users\HoJin\Desktop\python\ML_project\zreo_py\DL_Project\Trash-classification\ecosort\test\images\-4_jpg.rf.ff57349f27ca3ad85c3794266368d483.jpg: 640x640 1 Styrofoam, 17.0ms
image 4/225 c:\Users\HoJin\Desktop\python\ML_project\zreo_py\DL_Project\Trash-classification\ecosort\test\images\-7_jpg.rf.4a8ccd140c4ba931bb5f1ece10e8bc43.jpg: 640x640 1 General, 16.8ms
image 5/225 c:\Users\HoJin\Desktop\python\ML_project\zreo_py\DL_Project\Trash-classification\ecosort\test\images\KakaoTalk_20240318_201036576_15_jpg.rf.e8213b3b8450d429976e5501d263ae90.jpg: 640x640 1 Styrofoam, 17.0ms
image 6/225 c:\Users\HoJin\Desktop

In [7]:
dataDir = 'runs/detect/train2/'

In [8]:
bestModelpath = 'runs/detect/train/weights/best.pt'
bestModel = YOLO(bestModelpath)

In [10]:
import matplotlib.image as mpimg

def show_random_images(folder_path, num_images=9):
    # Get the list of image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    # If the number of images in the folder is less than requested, adjust num_images
    num_images = min(num_images, len(image_files))
    
    # Randomly select num_images images
    selected_images = random.sample(image_files, num_images)
    
    # Plot the selected images
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    for i, image_file in enumerate(selected_images):
        row = i // 3
        col = i % 3
        image_path = os.path.join(folder_path, image_file)
        image = mpimg.imread(image_path)
        axes[row, col].imshow(image)
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

# Example usage:
folder_path = 'train\images'
show_random_images(folder_path)

<Figure size 1000x1000 with 9 Axes>

In [19]:
bestModelpath = 'runs/detect/train/weights/best.pt'
bestModel = YOLO(bestModelpath)

In [21]:
import shutil
videoPath = 'test2.mp4'

bestModel.predict(source=videoPath, save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/102) c:\Users\HoJin\Desktop\python\ML_project\zreo_py\DL_Project\Trash-classification\ecosort\test2.mp4: 384x640 1 Plastic, 36.9ms
video 1/1 (frame 2/102) c:\Users\HoJin\Desktop\python\ML_project\zreo_py\DL_Project\Trash-classification\ecosort\test2.mp4: 384x640 1 Plastic, 30.4ms
video 1/1 (frame 3/102) c:\Users\HoJin\Desktop\python\ML_project\zreo_py\DL_Project\Trash-classification\ecosort\test2.mp4: 384x640 1 Plastic, 23.9ms
video 

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'General', 1: 'Glass', 2: 'Metal', 3: 'Paper', 4: 'Plastic', 5: 'Styrofoam', 6: 'Vinyl'}
 obb: None
 orig_img: array([[[126, 131, 137],
         [126, 131, 137],
         [126, 131, 137],
         ...,
         [173, 180, 181],
         [173, 180, 181],
         [173, 180, 181]],
 
        [[126, 131, 137],
         [126, 131, 137],
         [126, 131, 137],
         ...,
         [173, 180, 181],
         [173, 180, 181],
         [173, 180, 181]],
 
        [[128, 133, 139],
         [128, 133, 139],
         [128, 133, 139],
         ...,
         [173, 180, 181],
         [173, 180, 181],
         [173, 180, 181]],
 
        ...,
 
        [[ 42,  43,  48],
         [ 42,  43,  48],
         [ 42,  43,  48],
         ...,
         [155, 163, 167],
         [156, 164, 168],
         [156, 164, 168]],
 
        [[ 42,  43,  48],
     